# 从动作流（FlowAction）的设计思考函数的可组合性原理（草稿）

## -1. 关于主题

- **为什么要这么写？**
- 函数式编程

## 0. 关于动作流
- 前置动作，实现逻辑和后置动作，称为流(Flow)。
- 调用服务(InvokeService), 打开视图(OpenView), 关闭页面(Close), 跳转（Goto）称为动作（Action）
两者合起来称为 (FlowAction)
- 参考 [Action动作流程设计](https://aliyuque.antfin.com/trantor/eewi6i/hxr71y4p2ntenoih)

## 1. 动作流的Schema描述

### 前置节点 - Validate

In [1]:
const beforeFlow = {
    type: 'Flow',
    name: 'Validate',
    props: {
        // 这里只是演示，实际中不需要，直接实现成 breakOnFirst即可满足需求
        breakOnFirst: true,
    },
    children: [
        {
            type: 'Action',
            name: 'ValidateData',
            props: {
              // 规则选择器的属性值
            }
        },
        
        {
            type: 'Action',
            name: 'InvokeService',
            props: {
                serviceKey: 'xxx',
            }
        }
    ]
}

### 逻辑节点 - Compute

In [2]:
const computeFlow = {
    type: 'Flow',
    name: 'Compute',
    props: {
        parallel: true,
    },
    children: [
        {
            type: 'Action',
            name: 'InvokeService',
            props: {
                serviceKey: 'loadProducts',
            }
        },
        {
            // 工作流
            type: 'Action',
            name: 'InvokeFlow',
            props: {
                flowKey: 'notifyDingding'
            }
        }
    ]
}

### 后置节点 - Effect

In [3]:
const afterFlow = {
    type: 'Flow',
    name: 'Effect',
    props: {},
    children: [
        {
            type: 'Action',
            name: 'OpenView',
            props: {
                viewKey: 'sayHello'
            }
        }
    ]
}

### Pipeline

In [4]:
const pipeline = {
    type: 'Flow',
    name: 'Pipeline',
    children: [
        beforeFlow,
        computeFlow,
        afterFlow
    ]
}

In [5]:
pipeline


{
  type: 'Flow',
  name: 'Pipeline',
  children: [
    {
      type: 'Flow',
      name: 'Validate',
      props: [Object],
      children: [Array]
    },
    {
      type: 'Flow',
      name: 'Compute',
      props: [Object],
      children: [Array]
    },
    { type: 'Flow', name: 'Effect', props: {}, children: [Array] }
  ]
}


### 类型定义

In [6]:
type ActionNode = {
    type: 'Action',
    name: string,
    props: Record<string, unknown>
}

type FlowNode = {
    type: 'Flow',
    name: string,
    props: Record<string, unknown>
    children: (FlowNode | ActionNode)[]  // 这里支持嵌套
}

### 动作的定义

- 什么是动作？如何定义动作？
- 动作是一个操作，操作属于一种计算，函数即计算，所以动作可以用函数表达。—— 动作即函数

### trival version

In [ ]:
const validateData = (ctx, props) => {
    const record = ctx.record
    // 实际上会解析规则来执行
    return props.validate(record)
}



In [ ]:
const invokeService = async (ctx, props) => {
    const route = ctx.route
    return props.invokeService(props.serviceKey)
}

In [ ]:
const openView = (ctx, props) => {
  console.log('openView, openType: %s, recordId: %s', props.openType, ctx.record.id)
}

const openView = props => ctx => {
}

oepn({ type: 'modal' }) // ctx => {}

In [7]:
const ctx = {
    record: { id: '1', name: 'iphone' },
    route: { module: 'products', action: 'show' }
}

In [ ]:
openView(ctx, { openType: 'modal' })

### 问题和想法

- 函数和函数的组合
- [ ] 为什么从Schema的定义能想到函数的组合？

## 函数组合 Function Composition

### arity1

- [ ] 什么样的函数可组合？


In [8]:
const add1 = x => x + 1
const x2 = x => x * 2
const add = (x, y) => x + y

In [9]:
// (x + 1) * 2

const f = x => x2(add1(x))
f(100)

202


In [10]:
const compose = (f, g) => x => f(g(x)) 

In [11]:
const f = compose(x2, add1)

In [12]:
f(100)

202


### curry

- [ ] how to get arity1?

In [13]:
const add = (x, y, z) => x + y + z
const add2 = x => y => z => x + y + z

In [14]:
add(1,2,3)

6


In [16]:
const f = add2(1)(2)    
f(3)

6


### pointless

- [ ] 函数参数的顺序应该怎么样？

In [6]:
const range = n => [...Array(n).keys()]

In [18]:
range(10)

[
  0, 1, 2, 3, 4,
  5, 6, 7, 8, 9
]


In [7]:
const { faker } = require('@faker-js/faker')

[faker-js](https://fakerjs.dev/guide/usage.html) [faker-js/api](https://fakerjs.dev/api/)

In [20]:
faker.animal.cat()

Birman


In [8]:
const cats = range(100).map(_ => ({
    id: faker.datatype.uuid(),
    name: faker.animal.cat(),
    age: faker.datatype.number(100),
    dark: faker.datatype.boolean() 
}))
cats

[
  {
    id: '36eaf9f3-1955-4311-bcff-9a6b07958c2b',
    name: 'Ragdoll',
    age: 91,
    dark: true
  },
  {
    id: '0969a674-c79e-4d5b-a4b0-8454fe47806e',
    name: 'Russian Blue',
    age: 97,
    dark: false
  },
  {
    id: '89248fbd-2b5b-4745-9558-9188304435be',
    name: 'Snowshoe',
    age: 32,
    dark: false
  },
  {
    id: '1eb3617e-6bce-498a-b68f-626517848f16',
    name: 'Selkirk Rex',
    age: 1,
    dark: false
  },
  {
    id: '6436790b-1488-4774-9ec5-cff56085f6f8',
    name: 'Maine Coon',
    age: 62,
    dark: false
  },
  {
    id: '0bd7cfd3-73dd-4c84-94d7-f85a835847e1',
    name: 'British Shorthair',
    age: 2,
    dark: true
  },
  {
    id: '034efd3e-86d8-4c93-bc5c-803af63c09b5',
    name: 'Munchkin',
    age: 65,
    dark: true
  },
  {
    id: '159f3f54-c3dd-4975-b694-e3c790f46d4b',
    name: 'Turkish Angora',
    age: 47,
    dark: false
  },
  {
    id: '29654ead-9a05-4647-9987-6b9cf4ca9215',
    name: 'Norwegian Forest Cat',
    age: 31,
    dark: true
  

In [23]:
cats.filter(x => x.age < 10).map(x => x.id)

[
  'c45ef680-7d20-4980-a012-d9d86d869ecb',
  '97033e7c-4b3b-453c-9e63-25a60653a30e',
  '896c1373-c796-4b4a-9caa-c4b4dca101fb',
  '907d9f43-99db-40b8-80f8-e22c79817aca',
  '8cf7b5d4-27fb-4300-a717-960d1cc4ec73',
  'c8e94f13-d9b3-429b-849b-7840f1bef9fa',
  '459ba4d0-9569-4d60-a2e8-9aaaf0675451',
  'c682a4f8-e846-4cff-9dfa-b5ed20da0b93',
  '1fcabec5-5618-46d0-a10f-623198826eeb',
  '12aa4775-4f60-4802-84a2-a2020c4cb47c',
  'a11d6477-1598-4d12-8052-5f7ab8a728dc',
  'c2316009-d204-422a-8797-1be3b94e1b5f',
  'df835009-ab24-4821-b027-d64ef72ea772'
]


In [24]:
const findByAge = (list, age) => list.filter(x => x.age < age)

In [25]:
findByAge(cats, 10)

[
  {
    id: 'c45ef680-7d20-4980-a012-d9d86d869ecb',
    name: 'Sokoke',
    age: 0,
    dark: true
  },
  {
    id: '97033e7c-4b3b-453c-9e63-25a60653a30e',
    name: 'Thai',
    age: 4,
    dark: false
  },
  {
    id: '896c1373-c796-4b4a-9caa-c4b4dca101fb',
    name: 'Cornish Rex',
    age: 7,
    dark: true
  },
  {
    id: '907d9f43-99db-40b8-80f8-e22c79817aca',
    name: 'Snowshoe',
    age: 5,
    dark: false
  },
  {
    id: '8cf7b5d4-27fb-4300-a717-960d1cc4ec73',
    name: 'Bengal',
    age: 2,
    dark: false
  },
  {
    id: 'c8e94f13-d9b3-429b-849b-7840f1bef9fa',
    name: 'Oriental',
    age: 1,
    dark: false
  },
  {
    id: '459ba4d0-9569-4d60-a2e8-9aaaf0675451',
    name: 'Manx',
    age: 5,
    dark: true
  },
  {
    id: 'c682a4f8-e846-4cff-9dfa-b5ed20da0b93',
    name: 'Turkish Van',
    age: 0,
    dark: true
  },
  {
    id: '1fcabec5-5618-46d0-a10f-623198826eeb',
    name: 'Serengeti',
    age: 3,
    dark: true
  },
  {
    id: '12aa4775-4f60-4802-84a2-a2020c4c

In [28]:
const findByAge = list => age => list.filter(x => x.age < age)

//fiterWithAge(cats)(10)
findByAge(cats)   (10)
''

In [29]:
const search = list => text => list.filter(x => x.name.includes(text))

In [31]:
//search(cats)('S')
search(cats)('S')

[
  {
    id: '7acb6309-eccf-4c1b-bd3d-3c6f5dbcd124',
    name: 'Singapura',
    age: 46,
    dark: true
  },
  {
    id: '6f299695-28dd-4139-a9e2-f320f37c9b4d',
    name: 'British Shorthair',
    age: 89,
    dark: false
  },
  {
    id: '9ce922c4-db78-4f43-a150-21d985df1c97',
    name: 'Selkirk Rex',
    age: 41,
    dark: true
  },
  {
    id: '6462caac-3b47-426d-a25c-16af2f5900ee',
    name: 'Serengeti',
    age: 100,
    dark: true
  },
  {
    id: '5eeb866a-a819-4062-b7d5-7fc73bd90529',
    name: 'Sphynx',
    age: 70,
    dark: false
  },
  {
    id: '1ceb3133-f7dd-4122-934e-1e7cb8c9a453',
    name: 'Selkirk Rex',
    age: 75,
    dark: false
  },
  {
    id: 'a923bbab-8ccc-46f9-ac83-54ab76697510',
    name: 'Scottish Fold',
    age: 69,
    dark: false
  },
  {
    id: 'c45ef680-7d20-4980-a012-d9d86d869ecb',
    name: 'Sokoke',
    age: 0,
    dark: true
  },
  {
    id: '59d73da4-409d-4ce8-a2e1-81e01dc12ce8',
    name: 'Somali',
    age: 71,
    dark: false
  },
  {
    id: '4

In [32]:
const fiindByAgeAndSearch = (list, age, text) => search(findByAge(list)(age))(text)

In [34]:
 fiindByAgeAndSearch(cats, 10, 'S')
''

In [37]:
const findByAge = age => list => list.filter(x => x.age < age)
const search = text => list => list.filter(x => x.name.includes(text))

In [38]:
const fiindByAgeAndSearch = (age, text) => compose(search(text), findByAge(age))

In [ ]:
fiindByAgeAndSearch(10, 'S')(cats)

### datalast

In [39]:
const filter = f => list => list.filter(f)
const map = f => list => list.map(f)
const take = n => list => list.slice(0, n)

In [40]:
const compose = (a, b, c) => x => a(b(c(x))) 

In [42]:
const get2DarkCatsWithS = compose(
    take(4),
    filter(x => x.name.includes('S')),
    filter(x => x.dark),
)

In [43]:
get2DarkCatsWithS(cats)

[
  {
    id: '7acb6309-eccf-4c1b-bd3d-3c6f5dbcd124',
    name: 'Singapura',
    age: 46,
    dark: true
  },
  {
    id: '9ce922c4-db78-4f43-a150-21d985df1c97',
    name: 'Selkirk Rex',
    age: 41,
    dark: true
  },
  {
    id: '6462caac-3b47-426d-a25c-16af2f5900ee',
    name: 'Serengeti',
    age: 100,
    dark: true
  },
  {
    id: 'c45ef680-7d20-4980-a012-d9d86d869ecb',
    name: 'Sokoke',
    age: 0,
    dark: true
  }
]


## use ramda

In [44]:
const { map, filter, compose, prop, take, pick, sortBy } = require('ramda')

### transform data

In [46]:
cats
''

In [49]:

const transform = map(pick(['id', 'name']))
 transform(cats)
''

In [51]:
const getData = compose(
    take(5),
    map(pick(['id', 'name'])),
    sortBy(prop('age')),
    filter(x => x.age > 10)
)

getData(cats)
''

### curry

In [52]:
const add = a => b => c => a + b + c
add(1)(2)(3)

6


In [54]:
const { curry } = require('ramda')

In [55]:
const add = curry((a, b, c) => a + b + c)
add(1, 2, 3)
add(1, 2)            (3)
add(1)          (2, 3)
add(1)    (2)  (3)

6


## 组合动作函数

### schema node

In [ ]:
const beforeFlow = {
    type: 'Flow',
    name: 'Validate',
    props: {
        // 这里只是演示，实际中不需要，直接实现成 breakOnFirst即可满足需求
        breakOnFirst: true,
    },
    children: [
        {
            type: 'Action',
            name: 'ValidateData',
            props: {
              // 规则选择器的属性值
            }
        },
        
        {
            type: 'Action',
            name: 'InvokeService',
            props: {
                serviceKey: 'xxx',
            }
        }
    ]
}

In [ ]:
const computeFlow = {
    type: 'Flow',
    name: 'Compute',
    props: {
        parallel: true,
    },
    children: [
        {
            type: 'Action',
            name: 'InvokeService',
            props: {
                serviceKey: 'loadProducts',
            }
        },
        {
            // 工作流
            type: 'Action',
            name: 'InvokeFlow',
            props: {
                flowKey: 'notifyDingding'
            }
        }
    ]
}

### 动作的函数类型

#### 可组合: curry, datalast, arity 1

In [56]:
const invokeService = props => ctx => {
    
}

const invokeFlow = props => ctx => {
}

// or
const validateData = (a, b, c) => ctx => {
}

#### async

In [57]:
const sleep = n => new Promise(r => setTimeout(r, n))

const invokeService = props => async ctx => {
    await sleep(300)
    console.log('invoke service, props: %o, ctx: %o', props, ctx)
    return { serviceKey: props.serviceKey, param: props.param, route: ctx.route }
}

In [58]:
const invokeFlow = props => async ctx => {
    await sleep(300)
    console.log('invoke flow, props: %o, ctx: %o', props, ctx)
    return { flowKey: props.flowKey, route: ctx.route }
}

In [59]:
const ctx = {
    record: { id: '1', name: 'iphone' },
    route: { module: 'products', action: 'show' }
}

#### test

The code should be written that easy to test, but ideally it should not require testing.

In [60]:
await invokeService({ serviceKey: 'xxxbbb', param: '123' })(ctx)

invoke service, props: { serviceKey: 'xxxbbb', param: '123' }, ctx: {
  record: { id: '1', name: 'iphone' },
  route: { module: 'products', action: 'show' }
}
{
  serviceKey: 'xxxbbb',
  param: '123',
  route: { module: 'products', action: 'show' }
}


## 实现Flow

- 动作的组合

In [ ]:
const computeFlow = {
    type: 'Flow',
    name: 'Compute',
    props: {
        parallel: true,
    },
    children: [
        {
            type: 'Action',
            name: 'InvokeService',
            props: {
                serviceKey: 'loadProduct',
                param: 1,
            }
        },
        {
            type: 'Action',
            name: 'InvokeService',
            props: {
                serviceKey: 'loadProduct',
                param: 2,
            }
        },
        {
            // 工作流
            type: 'Action',
            name: 'InvokeFlow',
            props: {
                flowKey: 'notifyDingding'
            }
        }
    ]
}

函数即数据， 将上述结构转成函数

In [61]:
const Actions = {
  InvokeService: invokeService,
  InvokeFlow: invokeFlow
}

// dispatch method


In [62]:
Actions

{
  InvokeService: [Function: invokeService],
  InvokeFlow: [Function: invokeFlow]
}


In [63]:
computeFlow

{
  type: 'Flow',
  name: 'Compute',
  props: { parallel: true },
  children: [
    { type: 'Action', name: 'InvokeService', props: [Object] },
    { type: 'Action', name: 'InvokeFlow', props: [Object] }
  ]
}


In [64]:
const fns = computeFlow.children.map(child => Actions[child.name](child.props))

In [65]:
fns

[ [AsyncFunction (anonymous)], [AsyncFunction (anonymous)] ]


In [66]:
type Context = unknown
type ActionFn = (ctx: Context) => unknown

### Flow的函数类型

In [ ]:
type Dict = Record<string, unknown>
const compute = (props: Dict, children: ActionFn[]) => async (ctx: Context) => {
  
}

### 并行 & map

In [67]:
const compute = (props, children) => async ctx => {
    return Promise.all(children.map(fn => fn(ctx))) 
}

In [68]:
await compute({}, fns)(ctx)

invoke service, props: { serviceKey: 'loadProducts' }, ctx: {
  record: { id: '1', name: 'iphone' },
  route: { module: 'products', action: 'show' }
}
invoke flow, props: { flowKey: 'notifyDingding' }, ctx: {
  record: { id: '1', name: 'iphone' },
  route: { module: 'products', action: 'show' }
}
[
  {
    serviceKey: 'loadProducts',
    param: undefined,
    route: { module: 'products', action: 'show' }
  },
  {
    flowKey: 'notifyDingding',
    route: { module: 'products', action: 'show' }
  }
]


### 串行 & reduce

In [69]:
const compute = (props, children) => async ctx => {
    // for, push. pop,
    return children.reduce((promise, fn) => promise.then(x => fn({ ...ctx, ...x })), Promise.resolve({}))
}

In [70]:
await compute({}, fns)(ctx)

invoke service, props: { serviceKey: 'loadProducts' }, ctx: {
  record: { id: '1', name: 'iphone' },
  route: { module: 'products', action: 'show' }
}
invoke flow, props: { flowKey: 'notifyDingding' }, ctx: {
  record: { id: '1', name: 'iphone' },
  route: { module: 'products', action: 'show' },
  serviceKey: 'loadProducts',
  param: undefined
}
{
  flowKey: 'notifyDingding',
  route: { module: 'products', action: 'show' }
}


more reducer

In [71]:
// v => v
// (a, v) => a
const list = range(100).map(_ => Math.random())

In [ ]:
// list

In [72]:
list.reduce((acc, item) => acc + item, 0)

51.72279016685944


In [ ]:
// cats

In [74]:
// 组织成以age为key的Map列表

cats.reduce((acc, cat) => {
  const bag = acc[cat.age] || []
  acc[cat.age] = bag
  bag.push(cat)
  return acc
}, {})


{
  '0': [
    {
      id: 'c45ef680-7d20-4980-a012-d9d86d869ecb',
      name: 'Sokoke',
      age: 0,
      dark: true
    },
    {
      id: 'c682a4f8-e846-4cff-9dfa-b5ed20da0b93',
      name: 'Turkish Van',
      age: 0,
      dark: true
    },
    {
      id: 'a11d6477-1598-4d12-8052-5f7ab8a728dc',
      name: 'Burmese',
      age: 0,
      dark: true
    }
  ],
  '1': [
    {
      id: 'c8e94f13-d9b3-429b-849b-7840f1bef9fa',
      name: 'Oriental',
      age: 1,
      dark: false
    }
  ],
  '2': [
    {
      id: '8cf7b5d4-27fb-4300-a717-960d1cc4ec73',
      name: 'Bengal',
      age: 2,
      dark: false
    }
  ],
  '3': [
    {
      id: '1fcabec5-5618-46d0-a10f-623198826eeb',
      name: 'Serengeti',
      age: 3,
      dark: true
    }
  ],
  '4': [
    {
      id: '97033e7c-4b3b-453c-9e63-25a60653a30e',
      name: 'Thai',
      age: 4,
      dark: false
    },
    {
      id: 'c2316009-d204-422a-8797-1be3b94e1b5f',
      name: 'Exotic Shorthair',
      age: 4,
      dark:

In [79]:
const { groupBy, prop } = require('ramda')

groupBy(prop('age'), cats); //(cats); //(cats)
''

In [80]:
const { reduce } = require('ramda')

const add = (a, b) => a + b
reduce(add, 0, list)

51.72279016685944


In [81]:
const sumCatsAge = compose(
  reduce(add, 0),
  map(prop('age'))
)

sumCatsAge(cats)


5393


### break in reduce

In [82]:
const { reduced, reduce } = require('ramda')

In [83]:
const reducer = (a, b) => a > 1000 ? reduced(a) : a + b
reduce(reducer, 0)      (range(100))

// 如何实现呢？

1035


### compose

In [89]:
// a => b

const compose2 = (f, g) => x => f(g(x))
const compose = (...fns) => fns.reduce(compose2, x => x)
//reduceRight

[1,2,3,4].reduce

//1+2 +3 + 4
// compose(a, b, c) a(b(c(x)))

7:2 - Left side of comma operator is unused and has no side effects.
7:2 - Left side of comma operator is unused and has no side effects.
7:2 - Left side of comma operator is unused and has no side effects.


In [90]:
const add = a => b => a + b

compose(
  add(1),
  add(2),
  add(3),
  add(4)
)(10)

20


In [93]:
const compose = (...args) => reduce(compose2, x => x, args)

In [92]:
compose(
  compose(
    add(1),
    add(2),
  ),
  compose(
    compose(
      add(3),
      add(4),
    ),
    add(5)
  ),
  add(6)
)(0)

21


#### 结合率

In [99]:
const compose = (f, g, k) => x => f(g(k(x)))

In [96]:
const add1 = x => {
  console.log('add1')
  return x + 1
}
const add2 = x => {
  console.log('add2')
  return x + 1
}
const add3 = x => {
  console.log('add3')
  return x + 1
}

In [ ]:
// compose(add1, add2, add3)(10)

In [101]:
const compose2= (f, g, k) => x => k(g(f(x)))

In [102]:
compose2(add1, add2, add3)(10)

add1
add2
add3
13


In [ ]:
const compose = (...args) => args.reduce(compose2, x => x)

In [ ]:
const compose = (...args) => args.reduceRight(compose2, x => x)

In [ ]:
const add = a => b => a + b
compose(
  compose(
    add(1),
    add(2),
  ),
  compose(
    compose(
      add(3),
      add(4),
    ),
    add(5)
  ),
  add(6)
) (10)

- one line version of compose

In [ ]:
// const compose = (...fns) => (...args) => fns.reduceRight((acc, fn) => [fn.call(null, ...acc)], args)[0]

## 实现 Validation Flow

In [ ]:
const beforeFlow = {
    type: 'Flow',
    name: 'Validate',
    props: {
        // 这里只是演示，实际中不需要，直接实现成 breakOnFirst即可满足需求
        breakOnFirst: true,
    },
    children: [
        {
            type: 'Action',
            name: 'ValidateData',
            props: {
              // 规则选择器的属性值
            }
        },
        
        {
            type: 'Action',
            name: 'InvokeService',
            props: {
                serviceKey: 'xxx',
            }
        }
    ]
}

### Validate Flow的实现

In [103]:
const validateFlow = (props, actions) => async ctx => {
    for (const fn of actions) {
        const ret = await fn(ctx)
        if (!ret) {
          return false
        }
    }
    return true
}

In [104]:
ctx

{
  record: { id: '1', name: 'iphone' },
  route: { module: 'products', action: 'show' }
}


In [105]:
const beforeFlow = {
    type: 'Flow',
    name: 'Validate',
    props: {
        // 这里只是演示，实际中不需要，直接实现成 breakOnFirst即可满足需求
        breakOnFirst: true,
    },
    children: [
        {
            type: 'Action',
            name: 'ValidateData',
            props: {
                name: 'iphone'
            }
        },
        
        {
            type: 'Action',
            name: 'InvokeService',
            props: {
                serviceKey: 'xxx',
                id: '1'
            }
        }
    ]
}

In [106]:
const ValidateData = props => async ctx => ctx.record.name === props.name
const InvokeService = props => async ctx => ctx.record.id === props.id

const Actions = {
  ValidateData,
  InvokeService,
}

In [107]:
const fns = beforeFlow.children.map(child => Actions[child.name](child.props))

fns

[ [AsyncFunction (anonymous)], [AsyncFunction (anonymous)] ]


In [108]:
ctx

{
  record: { id: '1', name: 'iphone' },
  route: { module: 'products', action: 'show' }
}


In [109]:
await validateFlow(beforeFlow.props, fns)  (ctx)

true


## compile - 组合起来

### Schema

In [111]:
type FlowActionNode = ActionNode | FlowNode

type Dict = Record<string, unknown>

type ActionNode = {
    type: 'Action',
    name: string,
    props: Dict,
}

type FlowNode = {
    type: 'Flow',
    name: string,
    props: Dict,
    children: FlowActionNode[]
}

### API

#### 之前的定义

In [112]:
type Transformer = (ctx: Context) => Promise<unknown>

// type ActionFn = (props: Dict) => ctx => result
type ActionFn = (props: Dict) => Transformer
type FlowFn = (props: Dict, transformers: Transformer[]) => Transformer

#### 使用方法

In [ ]:

// 定义Action函数集 
const Actions: Record<string, ActionFn> = { }

// 定义流函数集
const Flows: Record<string, FlowFn> = {}

// 构建编译器
const compiler = createCompiler({ Actions, Flows })  // 只有这个没实现

// 创建转换器
const transformer = compiler(schemaNode)

// 使用
const ctx = {}
const result = await transformer(ctx)

### compiler的实现

- tree的应用
- 简单递归

### 思考类型

In [113]:
type Library = {
    Actions: Record<string, ActionFn>
    Flow: Record<string, FlowFn>
}

type CompilerFactory = (library: Library) => Compiler
type Compiler = (node: FlowActionNode) => Transformer

### 最简可运行实现

In [114]:
const createCompiler = ({ Actions, Flows }) => {
    const compiler = node => {
        console.log('compile', node)
        if (node.type !== 'Flow') {
            return Actions[node.name](node.props)
        }
        const transformers = node.children.map(compiler)
        return Flows[node.name](node.props, transformers)
    }
    return compiler
}

- pure
- curry
  - multi steps
- pointless
- recursion

#### 定义FlowAction

In [ ]:
const sleep = n => new Promise(r => setTimeout(r, n))

// 调用服务
const InvokeService = props => async ctx => {
    await sleep(300)
    console.log('invoke service, props: %o, ctx: %o', props, ctx)
    return { serviceKey: props.serviceKey, param: props.param, route: ctx.route }
}

const InvokeFlow = props => async ctx => {
    await sleep(300)
    console.log('invoke flow, props: %o, ctx: %o', props, ctx)
    return { flowKey: props.flowKey, route: ctx.route }
}

// 验证数据
const ValidateData = props => async ctx => ctx.record.name === props.name

// 验证服务
const ValidateService = props => async ctx => ctx.record.id === props.id

const Actions = {
    InvokeService,
    InvokeFlow,
    ValidateData,
    ValidateService,
}

const ValidateFlow = (props, transformers) => async ctx => {
    for (const fn of transformers) {
        const ret = await fn(ctx)
        if (!ret) {
          return false
        }
    }
    return true
}

const ComputeFlow = (props, transformers) => async ctx => {
    return transformers.reduce((promise, fn) => promise.then(x => fn({ ...ctx, ...x })), Promise.resolve({}))
}

const EffectFlow = (props, transformers) => async ctx => {
    await Promise.all(transformers.map(tranform => transform(ctx)))
    return null
}

const Flows = {
    Validate: ValidateFlow,
    Compute: ComputeFlow,
    Effect: EffectFlow,
    Pipeline: ValidateFlow,
}

### 使用

In [ ]:
const compiler = createCompiler({ Actions: Actions, Flows: Flows })

In [ ]:
const node = {
  type: 'Flow',
  name: 'Pipeline',
  children: [
      {
          type: 'Flow',
          name: 'Validate',
          children: [
              {
                  name: 'ValidateData',
                  props: { name: 'iphone' }
              },
              {
                  name: 'ValidateService',
                  props: { id: '123' }
              }
          ]
      },
      {
          type: 'Flow',
          name: 'Compute',
          children: [
              {
                  name: 'InvokeService',
                  props: { serviceKey: 'loadProduct', param: 1 }
              },
              
              {
                  name: 'InvokeService',
                  props: { serviceKey: 'loadProduct', param: 2 }
              },
              
              {
                  name: 'InvokeFlow',
                  props: { flowKey: 'xxxhhh' }
              },
          ]
      }
  ]
}


const transformer = compiler(node)

const ctx = {
    record: { id: '123', name: 'iphone' },
    route: { module: 'orders', action: 'show' },
}

console.log('1...')
await transformer(ctx)

console.log('2...')
await transformer(ctx)

## 待解决？

- [ ] `x` 和 `Promise.resolve(x)` 的区别
- [ ] type 和 typeclass

In [15]:
const { pipe, map, filter, tap, take, pick, compose } = require('ramda')

In [ ]:
//cats
//
x => Promise.resolve(x)

In [ ]:
await pipe(
    map(async x => ({ ...x, mate: faker.animal.bird() })),
    //tap(console.log),
    filter(x => x.age > 10),
) (cats)

''

In [ ]:
await pipe(
    map(async x => ({ ...x, mate: faker.animal.bird() })),
    //tap(console.log),
    // Defers,
    filter(x => x.age > 10),
) (cats)

In [ ]:
const Defers = list => ({
  filter: f => Defers(list.filter(x => x.then(f))),
  then: f => Promise.all(list).then(f),
})

### 执行流程和表达方式的分离

In [11]:
compose(
    take(5),
    map(pick(['id', 'name'])),
    filter(x => x.age > 50),
) (cats); //100

// cats.length

[
  { id: '36eaf9f3-1955-4311-bcff-9a6b07958c2b', name: 'Ragdoll' },
  { id: '0969a674-c79e-4d5b-a4b0-8454fe47806e', name: 'Russian Blue' },
  { id: '6436790b-1488-4774-9ec5-cff56085f6f8', name: 'Maine Coon' },
  { id: '034efd3e-86d8-4c93-bc5c-803af63c09b5', name: 'Munchkin' },
  { id: 'e5e573b0-6a75-42ff-a962-9149274c1563', name: 'Highlander' }
]


- [ ] 上述代码的问题是什么？

In [13]:
const { compose, transduce, append, flip } = require('ramda')

In [14]:
const tr = compose(
    filter(x => {
        console.log(x)
        return x.age > 50
    }),
    map(pick(['id', 'name'])),
    take(5)
)

//transormer: v=>v
// (acc, v) ->accr
reducer => reducer
transduce(tr, flip(append), [], cats)

{
  id: '36eaf9f3-1955-4311-bcff-9a6b07958c2b',
  name: 'Ragdoll',
  age: 91,
  dark: true
}
{
  id: '0969a674-c79e-4d5b-a4b0-8454fe47806e',
  name: 'Russian Blue',
  age: 97,
  dark: false
}
{
  id: '89248fbd-2b5b-4745-9558-9188304435be',
  name: 'Snowshoe',
  age: 32,
  dark: false
}
{
  id: '1eb3617e-6bce-498a-b68f-626517848f16',
  name: 'Selkirk Rex',
  age: 1,
  dark: false
}
{
  id: '6436790b-1488-4774-9ec5-cff56085f6f8',
  name: 'Maine Coon',
  age: 62,
  dark: false
}
{
  id: '0bd7cfd3-73dd-4c84-94d7-f85a835847e1',
  name: 'British Shorthair',
  age: 2,
  dark: true
}
{
  id: '034efd3e-86d8-4c93-bc5c-803af63c09b5',
  name: 'Munchkin',
  age: 65,
  dark: true
}
{
  id: '159f3f54-c3dd-4975-b694-e3c790f46d4b',
  name: 'Turkish Angora',
  age: 47,
  dark: false
}
{
  id: '29654ead-9a05-4647-9987-6b9cf4ca9215',
  name: 'Norwegian Forest Cat',
  age: 31,
  dark: true
}
{
  id: 'e5e573b0-6a75-42ff-a962-9149274c1563',
  name: 'Highlander',
  age: 52,
  dark: true
}
[
  { id: '36eaf9f3-

## S Expression & Lisp

- [ ] 数据和代码的关系

In [ ]:
const add = (x, y） => x + y

## typespec

In [ ]:
// (a, b) -> a

a -> b -> c

type Reducer = <A, B>(a: A, b: B) => A

In [ ]:
// a -> b
type Transformer = <A, B>(a: A) => B

In [ ]:
const add = (a, b, c) => a + b + c
const add = a => b => c => a + b + c
//add: a -> b -> c -> d

// reduce
// ((a, b) -> a) -> a -> [b] -> a